In [1]:
!pip -q install -U spacy
import spacy, sys
print("spaCy version:", spacy.__version__)

spaCy version: 3.8.7


In [24]:
from google.colab import files
uploaded = files.upload()

Saving training_data.json to training_data.json


In [25]:
import json, itertools

with open("training_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("Examples:", len(data))
print("Example 1:", data[0])

Examples: 16
Example 1: {'text': 'Apple Inc. reported revenue of $95 billion in Q2 2023.', 'entities': [[0, 10, 'ORG'], [31, 42, 'MONEY'], [46, 53, 'DATE']]}


In [26]:
import json
import spacy
from spacy.tokens import DocBin

with open("training_data.json", "r", encoding="utf-8") as f:
    TRAIN_DATA = json.load(f)

nlp = spacy.blank("en")
db = DocBin()

for record in TRAIN_DATA:
    doc = nlp.make_doc(record["text"])
    ents = []
    for start, end, label in record["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("train.spacy")
print("✅ Training data saved to train.spacy")


✅ Training data saved to train.spacy


In [27]:
import random, json
with open("training_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

random.shuffle(data)
split = int(0.8 * len(data))
train_data = data[:split]
dev_data = data[split:]

with open("train.json", "w", encoding="utf-8") as f: json.dump(train_data, f)
with open("dev.json", "w", encoding="utf-8") as f: json.dump(dev_data, f)

print("Train:", len(train_data), "Dev:", len(dev_data))


Train: 12 Dev: 4


In [42]:
def convert_to_spacy(input_json, output_file):
    import json, spacy
    from spacy.tokens import DocBin

    with open(input_json, "r", encoding="utf-8") as f:
        data = json.load(f)

    nlp = spacy.blank("en")
    db = DocBin()

    for record in data:
        doc = nlp.make_doc(record["text"])
        ents = []
        for start, end, label in record["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span:
                ents.append(span)
        doc.ents = ents
        db.add(doc)

    db.to_disk(output_file)

convert_to_spacy("training_data.json", "training_data.spacy")

In [44]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [45]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     60.58    7.37    5.04   13.73    0.07
 66     200        145.73   2384.82  100.00  100.00  100.00    1.00
157     400          0.03      0.02  100.00  100.00  100.00    1.00
257     600          0.00      0.00  100.00  100.00  100.00    1.00
357     800          0.02      0.01  100.00  100.00  100.00    1.00
538    1000        100.22     45.16  100.00  100.00  100.00    1.00
738    1200        106.58     10.42  100.00  100.00  100.00    1.00
938    1400         44.64      8.57  100.00  100.00  100.00    1.00
1138    1600         27.15      5.52

In [46]:
from google.colab import files
uploaded = files.upload()

Saving input_data.txt to input_data.txt


In [47]:
import spacy
from spacy import displacy

nlp_ner = spacy.load("model-best")

with open("input_data.txt", "r", encoding="utf-8") as f:
    text = f.read()

doc = nlp_ner(text)


displacy.render(doc, style="ent", jupyter=True)